In [ ]:
import tensorflow as tf
 
# Display the version
print(tf.__version__)    

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, Add, ReLU
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Model

2.12.0


In [ ]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

170498071/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
'''
Opción 1:
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=avg,
    classes=10,
    classifier_activation='softmax'
)

https://keras.io/api/applications/resnet/
'''

'\nOpción 1:\ntf.keras.applications.ResNet50(\n    include_top=True,\n    weights="imagenet",\n    input_tensor=None,\n    input_shape=None,\n    pooling=avg,\n    classes=10,\n    classifier_activation=\'softmax\'\n)\n\nhttps://keras.io/api/applications/resnet/\n'

In [ ]:
def identity_block(inp, filters: int):
    # Corresponde a los bloque normales, donde solo hay stripe=(1,1)
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same",kernel_initializer='he_normal')(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same",kernel_initializer='he_normal')(inp)
    x = BatchNormalization()(x)
    x = Add()([inp,x])
    x = Activation('relu')(x)
    return x

def conv_block(inp, filters: int):
    # Corresponde al primer bloque de cada grupo, el cual tiene stripe=(2,2)
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(2,2), padding="same",kernel_initializer='he_normal')(inp)
    x = BatchNormalization()(x)
    shortcut = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same",kernel_initializer='he_normal')(shortcut)
    x = BatchNormalization()(x)
    x = Add()([shortcut,x])
    x = Activation('relu')(x)
    return x

In [ ]:
# Poner semillas para demostrar mejora?

# ResNet-34
#Duda: No entiendo como se supone que se utiliza el stride = (2,2), pues hacer esto cambia las dimensiones del tensor.
# ¿Como esperan luego que sume los dos tensores de dimension diferente? Para arreglarlo, lo único que se me ocurrió
# (asumo está mal) es que, en caso de que haya stride=(2,2) ,la suma se realice entre el resultado de la primera convolusion que altera las dimensiones, 
# y lo que ocurra luego de hacer la segunda convolusión. 

# El input original para  ResNet-34 es de (224,224,3), mientras que el de Cifar-10 es de (32,32,3). ¿Qué hacemos para arreglarlo?

In [ ]:
i = Input(shape=x_train[0].shape)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same',kernel_initializer='he_normal')(i) # Originalmente ks=(7,7), stride=(2,2), f=64

x = Activation('relu')(x)
x = identity_block(x, 64)
x = identity_block(x, 64)
x = identity_block(x, 64)

x = conv_block(x, 128)
x = identity_block(x, 128)
x = identity_block(x, 128)
x = identity_block(x, 128)


x = conv_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)

x = conv_block(x, 512)
x = identity_block(x, 512)
x = identity_block(x, 512)

x = AveragePooling2D((2,2))(x) # Alterar esta para encajar en nuestro input
x = Flatten()(x)
x = Dense(10, activation='softmax')(x) # Originalmente es 1000, pero para cifar10 dee ser 10

model = Model(i, x)
 
# model description
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_132 (Conv2D)            (None, 32, 32, 64)   1792        ['input_5[0][0]']                
                                                                                                  
 activation_132 (Activation)    (None, 32, 32, 64)   0           ['conv2d_132[0][0]']             
                                                                                                  
 conv2d_134 (Conv2D)            (None, 32, 32, 64)   36928       ['activation_132[0][0]']         
                                                                                            

In [ ]:
# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Fit
r = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 67s 43ms/step - loss: 1.6867 - accuracy: 0.4341 - val_loss: 1.4612 - val_accuracy: 0.4935
Epoch 2/10
1563/1563 [==============================] - 68s 44ms/step - loss: 0.9486 - accuracy: 0.6691 - val_loss: 0.9089 - val_accuracy: 0.6843
Epoch 3/10
1563/1563 [==============================] - 69s 44ms/step - loss: 0.6962 - accuracy: 0.7597 - val_loss: 0.8804 - val_accuracy: 0.7199
Epoch 4/10
1563/1563 [==============================] - 69s 44ms/step - loss: 0.5391 - accuracy: 0.8147 - val_loss: 0.8103 - val_accuracy: 0.7727
Epoch 5/10
1563/1563 [==============================] - 68s 43ms/step - loss: 0.4143 - accuracy: 0.8568 - val_loss: 0.7764 - val_accuracy: 0.7511
Epoch 6/10
1563/1563 [==============================] - 69s 44ms/step - loss: 0.3006 - accuracy: 0.8947 - val_loss: 0.5533 - val_accuracy: 0.8216
Epoch 7/10
1563/1563 [==============================] - 72s 46ms/step - loss: 0.2082 - accuracy: 0.9274 - val_loss: 0.5633 -

In [ ]:
plt.plot(r.history['accuracy'], label='acc', color='red')
plt.plot(r.history['val_accuracy'], label='val_acc', color='green')
plt.legend()